# Analyse Exploratoire

### Import des modules

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

### Analyse Exploratoire

In [2]:
building_consumption = pd.read_csv('2016_Building_Energy_Benchmarking.csv')

FileNotFoundError: [Errno 2] No such file or directory: '2016_Building_Energy_Benchmarking.csv'

In [ ]:
# On regarde comment un batiment est défini dans ce jeu de données 
building_consumption.head()

In [ ]:
data=building_consumption

In [ ]:
# On regarde le nombre de valeurs manquantes par colonne ainsi que leur type 
data.info()

In [ ]:
data.nunique()

In [ ]:
# J'enlève les colonnes inutiles
data=data.drop(columns=['Electricity(kWh)','NaturalGas(therms)']) # redondant avec les kBtu qui sont utilisés dans d'autres colonnes
data=data.drop(columns=['Comments']) # Pas de données
data=data.drop(columns=['DataYear','City','State']) # 2016 à Seattle...

In [ ]:
data

In [ ]:
data.info()

Vu que tous les bâtiments ont un OSEBuildingID, je retire toutes les autres infos liées au bâtiments qui ne me sont pas utiles pour la mission (nom, adresse...)

In [ ]:
data=data.drop(columns=['PropertyName','Address','ZipCode','TaxParcelIdentificationNumber','CouncilDistrictCode'])

In [ ]:
#Je crée un df avec tous les outliers pour pouvoir les étudier ensuite
outlier=data.loc[~data['Outlier'].isna()]
outlier.shape

### BuildingType

In [ ]:
data.groupby(by=['BuildingType']).count()

In [ ]:
data.groupby(by=['BuildingType']).count()

Je devrais conserver seulement les bâtiments Non-résidentiels. Mais avant je dois vérifier qu'il n'y a pas d'erreur d'attribution. En étudiant les outliers notamment.

In [ ]:
data.groupby(by=['BuildingType']).describe()

In [ ]:
data[['BuildingType','Electricity(kBtu)']].groupby(by=['BuildingType']).describe()

In [ ]:
outlier.loc[outlier['BuildingType']=='Multifamily HR (10+)']

La seule ligne est un Low Outlier, je considère qu'il est bien attibué

In [ ]:
outlier[['BuildingType','Electricity(kBtu)','Outlier']].loc[outlier['BuildingType']=='Multifamily LR (1-4)']

In [ ]:
outlier[['BuildingType','Electricity(kBtu)','Outlier']].loc[outlier['BuildingType']=='Multifamily MR (5-9)']

In [ ]:
outlier[['BuildingType','Electricity(kBtu)','Outlier']].loc[outlier['BuildingType']=='SPS-District K-12']

#### TERMINER L'ANALYSE EXPLORATOIRE 

A réaliser : 
- Une analyse descriptive des données, y compris une explication du sens des colonnes gardées, des arguments derrière la suppression de lignes ou de colonnes, des statistiques descriptives et des visualisations pertinentes.

Qelques pistes d'analyse : 

* Identifier les colonnes avec une majorité de valeurs manquantes ou constantes en utilisant la méthode value_counts() de Pandas
* Mettre en evidence les différences entre les immeubles mono et multi-usages
* Utiliser des pairplots et des boxplots pour faire ressortir les outliers ou des batiments avec des valeurs peu cohérentes d'un point de vue métier 

Pour vous inspirer, ou comprendre l'esprit recherché dans une analyse exploratoire, vous pouvez consulter ce notebook en ligne : https://www.kaggle.com/code/pmarcelino/comprehensive-data-exploration-with-python. Il ne s'agit pas d'un modèle à suivre à la lettre ni d'un template d'analyses attendues pour ce projet. 

# Modélisation 

### Import des modules 

In [ ]:
#Selection
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV, 
    cross_validate,
)
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error 
from sklearn.inspection import permutation_importance

#Preprocess
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

#Modèles
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor


### Feature Engineering

A réaliser : Enrichir le jeu de données actuel avec de nouvelles features issues de celles existantes. 

En règle générale : On utilise la méthode .apply() de Pandas pour créer une nouvelle colonne à partir d'une colonne existante. N'hésitez pas à regarder les exemples dans les chapitres de cours donnés en ressource

In [ ]:
# CODE FEATURE ENGINEERING

### Préparation des features pour la modélisation

A réaliser :
* Si ce n'est pas déjà fait, supprimer toutes les colonnes peu pertinentes pour la modélisation.
* Tracer la distribution de la cible pour vous familiariser avec l'ordre de grandeur. En cas d'outliers, mettez en place une démarche pour les supprimer.
* Débarrassez-vous des features redondantes en utilisant une matrice de corrélation de Pearson. Pour cela, utiisez la méthode corr() de Pandas, couplé d'un graphique Heatmap de la librairie Seaborn 
* Réalisez différents graphiques pour comprendre le lien entre vos features et la target (boxplots, scatterplots, pairplot si votre nombre de features numériques n'est pas très élevé).
*  Séparez votre jeu de données en un Pandas DataFrame X (ensemble de feautures) et Pandas Series y (votre target).
* Si vous avez des features catégorielles, il faut les encoder pour que votre modèle fonctionne. Les deux méthodes d'encodage à connaitre sont le OneHotEncoder et le LabelEncoder

In [ ]:
# CODE PREPARATION DES FEATURES

### Comparaison de différents modèles supervisés

A réaliser :
* Pour chaque algorithme que vous allez tester, vous devez :
    * Réaliser au préalable une séparation en jeu d'apprentissage et jeu de test via une validation croisée.
    * Si les features quantitatives que vous souhaitez utiliser ont des ordres de grandeur très différents les uns des autres, et que vous utilisez un algorithme de regression qui est sensible à cette différence, alors il faut réaliser un scaling (normalisation) de la donnée au préalable.
    * Entrainer le modèle sur le jeu de Train
    * Prédire la cible sur la donnée de test (nous appelons cette étape, l'inférence).
    * Calculer les métriques de performance R2, MAE et RMSE sur le jeu de train et de test.
    * Interpréter les résultats pour juger de la fiabilité de l'algorithme.
* Vous pouvez choisir par exemple de tester un modèle linéaire, un modèle à base d'arbres et un modèle de type SVM
* Déterminer le modèle le plus performant parmi ceux testés.

In [1]:
# CODE COMPARAISON DES MODELES

### Optimisation et interprétation du modèle

A réaliser :
* Reprennez le meilleur algorithme que vous avez sécurisé via l'étape précédente, et réalisez une GridSearch de petite taille sur au moins 3 hyperparamètres.
* Si le meilleur modèle fait partie de la famille des modèles à arbres (RandomForest, GradientBoosting) alors utilisez la fonctionnalité feature importance pour identifier les features les plus impactantes sur la performance du modèle. Sinon, utilisez la méthode Permutation Importance de sklearn.

In [ ]:
# CODE OPTIMISATION ET INTERPRETATION DU MODELE